<a href="https://colab.research.google.com/github/muitagabi/mba_tcc/blob/main/%5BUSP%5D_Thesis_%7C_NLP_Code_%7C_STS_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installing Packages

In [ ]:
# Installing packages
!pip install pdfplumber
!pip install -U spacy
!python -m spacy download en_core_web_sm
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-03-03 21:08:51.741751: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-03 21:08:51.742388: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-03 21:08:51.742412: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned

#Pre-processing

In [ ]:
# Importing libraries
import pdfplumber
import spacy

# Solving issue with character encoding
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
# Saving SDG declaration localy
!curl "https://raw.githubusercontent.com/gabriellecastilho/mba_tcc/02b07d4a4f8da81f0de5cad7d576db161ed2cea7/SDG%20Declaration.pdf" > sdg_declaration.pdf
!pdfplumber < sdg_declaration.pdf > sdg_declaration.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  128k  100  128k    0     0   837k      0 --:--:-- --:--:-- --:--:--  837k


In [ ]:
# Saving SDG declaration by pages
sdg_content = []
keep = list("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 ")

with pdfplumber.open("sdg_declaration.pdf") as pdf:
    for i, page in enumerate(pdf.pages):
        page_ = pdf.pages[i]
        page_ = page_.extract_text(x_tolerance=3, y_tolerance=3, layout=False, x_density=7.25, y_density=13)
        page_ = " ".join(page_.lower().split('\n')) # delete '\n'
        page_ = "".join(item for item in page_ if item in keep) # delete non-alphanumeric characters mantaining space
        page_ = " ".join(page_.split()) # delete double spacing
        sdg_content.append(page_)

# Combining SDG 17 in one page
sdg_content[16] = "".join(sdg_content[-2:])
sdg_content.pop(-1)

# Compiling SDG declaration in one
sdg = " ".join(sdg_content)

# Specific cleaning for TF-IDF
nlp = spacy.load("en_core_web_sm")
sdg_content_tfidf = [" ".join(token.lemma_ for token in nlp(item) if not token.is_stop) for item in sdg_content] #lemmatizing and removing stop words
sdg_tfidf = " ".join(sdg_content_tfidf)

In [ ]:
# Saving company's sustainability report localy
!curl "https://rs.klabin.com.br/documents/785690889/974254204/klabin_RS2021_EN.pdf/802cfc0f-e31d-5b6b-a1e1-455755bd2a58?t=1673548244292" > sustainability_report.pdf
!pdfplumber < sustainability_report.pdf > sustainability_report.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18.5M  100 18.5M    0     0  12.1M      0  0:00:01  0:00:01 --:--:-- 12.1M


In [ ]:
# Saving sustainability report by pages
report_content = []
keep = list("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 ")

with pdfplumber.open("sustainability_report.pdf") as pdf:
    for i, page in enumerate(pdf.pages):
        page_ = pdf.pages[i]
        page_ = page_.extract_text(x_tolerance=3, y_tolerance=3, layout=False, x_density=7.25, y_density=13)
        page_ = " ".join(page_.lower().split('\n')) # delete '\n'
        page_ = "".join(item for item in page_ if item in keep) # delete non-alphanumeric characters mantaining space
        page_ = " ".join(page_.split()) # delete double spacing
        report_content.append(page_)

# Compiling sustainability report in one
report = " ".join(report_content)

# Specific cleaning for TF-IDF
report_content_tfidf = [" ".join(token.lemma_ for token in nlp(item) if not token.is_stop) for item in report_content] #lemmatizing and removing stop words
report_tfidf = " ".join(report_content_tfidf)

#TF-IDF

In [ ]:
# Importing libraries
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
for i, goal in enumerate(sdg_content_tfidf):
    corpus = [report_tfidf, goal]
    vect = TfidfVectorizer(min_df=1, stop_words="english")
    tfidf = vect.fit_transform(corpus)
    pairwise_similarity = tfidf * tfidf.T
    print(f"Goal {i+1}: {round(pairwise_similarity.toarray()[0,1] * 100, 2)}%")

Goal 1: 8.79%
Goal 2: 11.71%
Goal 3: 8.43%
Goal 4: 9.02%
Goal 5: 7.37%
Goal 6: 11.21%
Goal 7: 8.65%
Goal 8: 10.54%
Goal 9: 11.64%
Goal 10: 8.25%
Goal 11: 12.9%
Goal 12: 18.0%
Goal 13: 9.54%
Goal 14: 10.44%
Goal 15: 13.71%
Goal 16: 7.0%
Goal 17: 11.39%


#USE

In [ ]:
# Importing libraries
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#Loading USE pre-trained model
use_module_url = 'https://tfhub.dev/google/universal-sentence-encoder/4'
use_model_flow = hub.load(use_module_url)

report_use_emb = use_model_flow([report]).numpy() # report embedding

for i, goal in enumerate(sdg_content):
    goal_use_emb = use_model_flow([goal]).numpy() #goal embedding
    cos_sim_use = cosine_similarity(report_use_emb, goal_use_emb)[0][0] #calculating cosine similarity
    print(f"Goal {i+1}: {round(cos_sim_use * 100, 2)}")

Goal 1: 31.85
Goal 2: 44.14
Goal 3: 37.62
Goal 4: 31.85
Goal 5: 31.13
Goal 6: 37.95
Goal 7: 33.17
Goal 8: 37.97
Goal 9: 31.76
Goal 10: 25.17
Goal 11: 40.0
Goal 12: 41.44
Goal 13: 37.24
Goal 14: 42.97
Goal 15: 49.25
Goal 16: 31.3
Goal 17: 43.39


#SBERT Bi-Encoder

In [ ]:
# Importing libraries
from sentence_transformers import SentenceTransformer

In [ ]:
#Loading SBERT pre-trained model

#sbert_model = SentenceTransformer('sentence-transformers/msmarco-MiniLM-L6-cos-v5')
#sbert_model = SentenceTransformer('sentence-transformers/msmarco-MiniLM-L12-cos-v5')
#sbert_model = SentenceTransformer('sentence-transformers/msmarco-distilbert-cos-v5')
sbert_model = SentenceTransformer('sentence-transformers/msmarco-distilbert-dot-v5')

report_sbert_emb = sbert_model.encode([report], show_progress_bar=False) # report embedding

for i, goal in enumerate(sdg_content):
    goal_sbert_emb = sbert_model.encode([goal], show_progress_bar=False) #goal embedding
    cos_sim_sbert = cosine_similarity(report_sbert_emb, goal_sbert_emb)[0][0] #calculating cosine similarity
    print(f"Goal {i+1}: {round(cos_sim_sbert * 100, 2)}")

Goal 1: 70.02
Goal 2: 76.4
Goal 3: 71.25
Goal 4: 72.07
Goal 5: 68.63
Goal 6: 74.71
Goal 7: 74.42
Goal 8: 78.34
Goal 9: 77.21
Goal 10: 70.4
Goal 11: 74.59
Goal 12: 79.0
Goal 13: 72.84
Goal 14: 75.14
Goal 15: 75.29
Goal 16: 70.78
Goal 17: 75.66


#SimCSE

In [ ]:
### SUPERVISED ###
#Loading SimCSE pre-trained model

#sim_model_sup = SentenceTransformer('princeton-nlp/sup-simcse-roberta-large')
sim_sup_model = SentenceTransformer('princeton-nlp/sup-simcse-bert-large-uncased')

report_sim_sup_emb = sim_sup_model.encode([report], show_progress_bar=False) # report embedding

for i, goal in enumerate(sdg_content):
    goal_sim_sup_emb = sim_sup_model.encode([goal], show_progress_bar=False) #goal embedding
    cos_sim_sup = cosine_similarity(report_sim_sup_emb, goal_sim_sup_emb)[0][0] #calculating cosine similarity
    print(f"Goal {i+1}: {round(cos_sim_sup * 100, 2)}")

Goal 1: 75.3
Goal 2: 79.16
Goal 3: 72.75
Goal 4: 70.09
Goal 5: 65.35
Goal 6: 77.54
Goal 7: 75.86
Goal 8: 76.28
Goal 9: 77.65
Goal 10: 73.44
Goal 11: 76.23
Goal 12: 83.42
Goal 13: 74.55
Goal 14: 76.95
Goal 15: 82.51
Goal 16: 69.49
Goal 17: 77.54


In [ ]:
### UNSUPERVISED ###
#Loading SimCSE pre-trained model

sim_unsup_model = SentenceTransformer('princeton-nlp/unsup-simcse-roberta-large')

report_sim_unsup_emb = sim_unsup_model.encode([report], show_progress_bar=False) # report embedding

for i, goal in enumerate(sdg_content):
    goal_sim_unsup_emb = sim_unsup_model.encode([goal], show_progress_bar=False) #goal embedding
    cos_sim_unsup = cosine_similarity(report_sim_unsup_emb, goal_sim_unsup_emb)[0][0] #calculating cosine similarity
    print(f"Goal {i+1}: {round(cos_sim_unsup * 100, 2)}")

Goal 1: 42.5
Goal 2: 52.26
Goal 3: 41.07
Goal 4: 26.91
Goal 5: 33.93
Goal 6: 53.88
Goal 7: 47.65
Goal 8: 38.31
Goal 9: 44.86
Goal 10: 40.77
Goal 11: 42.64
Goal 12: 59.77
Goal 13: 45.57
Goal 14: 58.5
Goal 15: 59.93
Goal 16: 37.01
Goal 17: 50.33


#Ada

In [ ]:
#Reference: https://openai.com/blog/new-and-improved-embedding-model

In [ ]:
import openai
response = openai.Embedding.create(
  input="porcine pals say",
  model="text-embedding-ada-002"
)